## PCR

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = 'data/' 
file = 'MTBLS24.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 106 TOTAL PEAKS: 701
Done!


### 3. Extract X & Y

In [5]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTable2[peaklist]                                    
#XTlog = np.log(XT)                                          
#XTscale = cb.utils.scale(XTlog, method='auto')   
XTscale = np.array(XT)
XTknn = cb.utils.knnimpute(XTscale, k=3)                                      
                                         

### 4. Hyperparameters optimisation

In [7]:
# param_dict
#param_dict = {'n_components': [1,2,3,4,5,6]}                   
#param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12]} 
param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]} 

# Initalise
cv = cb.cross_val.kfold(model=cb.model.PCR,                      
                                X=XTknn,                                 
                                Y=Y,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=100)                                

# Run and plot
cv.run()  
cv.plot(metric='r2q2', ci=95)

Number of cores set to: 8
Running ...


100%|██████████| 3000/3000 [01:06<00:00, 44.78it/s]


Time taken: 1.12 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [8]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.PCR(n_components=5)
model.train(XTknn, Y)
model.test(XTknn)

# Evaluate 
model.evaluate(cutoffscore=0.5, plot_median=False) 

Loading BokehJS ...

In [9]:
# Bootstrap evaluate

model.booteval(XTknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:01<00:00, 84.85it/s]


Loading BokehJS ...

### 7. Save table

In [10]:
home = 'tables/'
file = 'PCR_MTBLS24_NH.xlsx'

model.save_table(home + file)

Done! Saved table as tables/PCR_MTBLS24_NH.xlsx
